In [1]:
#!git clone https://github.com/ait-aecid/clue-lds.git

Cloning into 'clue-lds'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 57 (delta 30), reused 27 (delta 12), pack-reused 0
Receiving objects: 100% (57/57), 31.30 KiB | 3.91 MiB/s, done.
Resolving deltas: 100% (30/30), done.


In [1]:
cd clue-lds/

/home/jupyter/clue-lds


In [4]:
#!wget https://zenodo.org/record/7119953/files/clue.zip

--2023-03-05 12:26:17--  https://zenodo.org/record/7119953/files/clue.zip
Resolving zenodo.org (zenodo.org)... 188.185.124.72
Connecting to zenodo.org (zenodo.org)|188.185.124.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 635105552 (606M) [application/octet-stream]
Saving to: ‘clue.zip’

clue.zip            100%[===================>] 605.68M   121MB/s    in 5.9s    

2023-03-05 12:26:23 (102 MB/s) - ‘clue.zip’ saved [635105552/635105552]



In [5]:
#!unzip clue.zip

Archive:  clue.zip
  inflating: clue.json               


# Get User Similarity

The next step is to compute similarities between pairs of users so that behavior patterns of users that are selected for switching are not too similar (which would make detection very difficult) and not too different (which would make detection trivial). Running the following script generates a similarity matrix sim.txt as well as the file user_info.txt containing user information that is necessary for the subsequent steps.

Running script:

user@ubuntu:~/clue-lds$ python3 get_user_similarity.py

In [2]:
import json
from dateutil import parser
import datetime
import pytz
import argparse

# argparser = argparse.ArgumentParser(description='Similarity computation.')
# argparser.add_argument('-d','--min_days', help='Minimum amount of days user must be active.', required=False)
# argparser.add_argument('-c','--w_count', help='Weight of average user event count per day for similarity computation.', required=False)
# argparser.add_argument('-e','--w_events', help='Weight of shared event types for similarity computation.', required=False)

min_days = 25
w_count = 0.3
w_events = 0.7

# args = vars(argparser.parse_args())
# if args["min_days"] is not None:
#     min_days = int(args["min_days"])
# if args["w_count"] is not None:
#     w_count = float(args["w_count"])
# if args["w_events"] is not None:
#     w_events = float(args["w_events"])

# Normalize weights in case that they do not sum to 1
w_sum = w_count + w_events
w_count /= w_sum
w_events /= w_sum

users = {}
cnt = 0
total_lines = 50522931

Original fields:

* uid - Unique anonymized identifier for the user generating the event, e.g., old-pink-crane-sharedealer.
* type - The action carried out by the user, e.g., file_accessed.
* time - Time stamp of the event in ISO format, e.g., 2021-01-01T00:00:02Z.

Derrived fields:

* actions
* cnt - += 1
* first - ts.timestamp()
* last - ts.timestamp()
* day_list - .add(datetime.datetime(ts.year, ts.month, ts.day, 0, 0, 0, 0, pytz.UTC).timestamp())
* days - len(users[uid]['day_list'])

In [3]:
# Go through all lines and retrieve information on executed events for each user
with open('clue.json') as f:
    for line in f:
        cnt += 1
        if int(cnt % (total_lines / 20)) == 0:
            print(str(int(cnt*100/total_lines)) + '%', end=' ', flush=True)
        j = json.loads(line)
        uid = j['uid']
        action = j['type']
        ts = parser.isoparse(j['time'])
        if uid not in users:
            users[uid] = {'actions': set([action]), 'cnt': 1, 'first': ts.timestamp(), 'last': ts.timestamp(), 'day_list': set([datetime.datetime(ts.year, ts.month, ts.day, 0, 0, 0, 0, pytz.UTC).timestamp()]), 'days': 1}
        else:
            users[uid]['actions'].add(action)
            users[uid]['cnt'] += 1
            users[uid]['last'] = ts.timestamp()
            users[uid]['day_list'].add(datetime.datetime(ts.year, ts.month, ts.day, 0, 0, 0, 0, pytz.UTC).timestamp())
            users[uid]['days'] = len(users[uid]['day_list'])
print('')

5% 10% 15% 20% 25% 30% 35% 40% 45% 50% 55% 60% 65% 70% 75% 80% 85% 90% 95% 100% 


In [4]:
users['yellow-chocolate-carp-busdriver']

{'actions': {'admin_settings_changed',
  'deleted_from_trashbin',
  'file_accessed',
  'file_created',
  'file_deleted',
  'file_renamed',
  'file_updated',
  'file_written',
  'group_created',
  'group_deleted',
  'groupadmin_removed_from_group',
  'login_attempt',
  'login_successful',
  'logout_occured',
  'password_changed',
  'permission_changed',
  'public_share_accessed',
  'public_share_expiration_date_changed',
  'public_share_password_changed',
  'quota_changed',
  'shared_link',
  'shared_user',
  'unshared_link',
  'unshared_user',
  'user_added_to_group',
  'user_became_groupadmin',
  'user_created',
  'user_data_viewed',
  'user_deleted',
  'user_removed_from_group'},
 'cnt': 19079,
 'first': 1499417877.0,
 'last': 1548405313.0,
 'day_list': {1499385600.0,
  1499644800.0,
  1499731200.0,
  1499817600.0,
  1499904000.0,
  1500249600.0,
  1500336000.0,
  1500422400.0,
  1500508800.0,
  1500595200.0,
  1500854400.0,
  1500940800.0,
  1501113600.0,
  1501200000.0,
  150137280

In [5]:
# Compute similarities between pairs of users and store them in a similarity matrix
similarity_matrix = {}
user_list = []
for user in users:
    users[user]['similarities'] = [] # use list instead of dict to reduce resulting file size
    user_list.append(user) # this list is necessary to map user names to list of similarities
    for user_inner in users:
        # Compute similarity based on average number of generated events
        cnt_sim = min(users[user]['cnt'] / users[user]['days'], users[user_inner]['cnt'] / users[user_inner]['days']) / max(users[user]['cnt'] / users[user]['days'], users[user_inner]['cnt'] / users[user_inner]['days'])
        # Compute similarity based on common event types
        action_sim = len(users[user]['actions'].intersection(users[user_inner]['actions'])) / len(users[user]['actions'].union(users[user_inner]['actions']))
        # Compute weighted similarity score
        sim_score = w_count * cnt_sim + w_events * action_sim
        users[user]['similarities'].append(round(sim_score, 3)) # round to decrease resulting file size
        if users[user]['days'] > min_days and users[user_inner]['days'] > min_days:
            if user not in similarity_matrix:
                similarity_matrix[user] = {}
            similarity_matrix[user][user_inner] = sim_score

In [6]:
print(w_count)
print(cnt_sim)
print(w_events)
print(action_sim)
print(w_count * cnt_sim + w_events * action_sim)

0.3
1.0
0.7
1.0
1.0


In [7]:
similarity_matrix['yellow-chocolate-carp-busdriver']

{'yellow-chocolate-carp-busdriver': 1.0,
 'renewed-emerald-hare-productionplanner': 0.24675612728206797,
 'green-white-frog-chimneysweep': 0.7211122189095898,
 'logical-coral-pig-warehouseman': 0.46386801566011426,
 'developed-harlequin-falcon-radiodirector': 0.46462127706226997,
 'integral-turquoise-narwhal-rugmaker': 0.24922551018250064,
 'labour-crimson-donkey-golfcaddy': 0.321600538552067,
 'ready-silver-angelfish-quarryworker': 0.48100081896133196,
 'equivalent-moccasin-sheep-therapist': 0.5481834148982236,
 'nice-orange-centipede-taxidriver': 0.4833618001016687,
 'central-silver-slug-zookeeper': 0.45682033019951995,
 'little-amber-minnow-reporter': 0.5242858516312444,
 'changing-yellow-landfowl-mastermariner': 0.37797138802433317,
 'repulsive-violet-canidae-warehousewoman': 0.21724922464433394,
 'sharp-pink-swan-hygienist': 0.2949535049607921,
 'marvellous-amaranth-fowl-accountant': 0.4516413250535122,
 'marginal-harlequin-pigeon-softwareconsultant': 0.13571607200479419,
 'shared

Saving output:

In [8]:
# Output the similarity matrix
with open('sim.txt', 'w+') as out:
    for user in similarity_matrix:
        string = ""
        for user_inner in similarity_matrix[user]:
            string += str(similarity_matrix[user][user_inner]) + ','
        out.write(string[:-1] + '\n')

# Convert sets to lists to enable serialization
for user in users:
    users[user]['actions'] = list(users[user]['actions'])
    users[user]['day_list'] = list(users[user]['day_list'])

# Output detailed information for further processing
with open('user_info.txt', 'w+') as out:
    out.write(json.dumps({'user_list': user_list, 'user_info': users}))    

# Generate test file

user@ubuntu:~/clue-lds$ python3 generate_test_file.py

Changing user pair with similarity 0.24:
 * User competent-aqua-hare-buildinginspector changed at 2022-09-28 00:00:00+00:00 (user originally carried out 1521 total events and 16 unique events during 59 active days).
 * User shared-fuchsia-cardinal-buildingadvisor changed at 2022-09-28 00:00:00+00:00 (user originally carried out 6356739 total events and 24 unique events during 1910 active days).
Changing user pair with similarity 0.15:
 * User dull-amethyst-buzzard-ledgerclerk changed at 2018-06-10 00:00:00+00:00 (user originally carried out 5328776 total events and 5 unique events during 291 active days).
 * User japanese-yellow-pike-thermalengineer changed at 2018-06-11 00:00:00+00:00 (user originally carried out 96841 total events and 23 unique events during 669 active days).
...

In [9]:
import json
from dateutil import parser
import random
import copy
import datetime
import pytz
import argparse

seed = 1234
num_pairs = 50
min_similarity = 0.1
max_similarity = 0.5
min_days_before_change = 25
min_days_after_change = 2
min_cnt = 100
min_unique_events = 4

# argparser = argparse.ArgumentParser(description='Generation.')
# argparser.add_argument('-s','--seed', help='Random generator seed.', required=False)
# argparser.add_argument('-p','--pairs', help='Number of switched users.', required=False)
# argparser.add_argument('-m','--min_sim', help='Minimum similarity for switching.', required=False)
# argparser.add_argument('-n','--max_sim', help='Maximum similarity for switching.', required=False)
# argparser.add_argument('-b','--days_before', help='Minimum active days of user before switchting.', required=False)
# argparser.add_argument('-a','--days_after', help='Minimum active days of user after switching.', required=False)
# argparser.add_argument('-c','--min_count', help='Minimum total number of events by switched user.', required=False)
# argparser.add_argument('-u','--min_unique', help='Minimum unique events by switched user.', required=False)

# args = vars(argparser.parse_args())
# if args["seed"] is not None:
#     seed = int(args["seed"])
# if args["pairs"] is not None:
#     num_pairs = int(args["pairs"])
# if args["min_sim"] is not None:
#     min_similarity = float(args["min_sim"])
# if args["max_sim"] is not None:
#     max_similarity = float(args["max_sim"])
# if args["days_before"] is not None:
#     min_days_before_change = int(args["days_before"])
# if args["days_after"] is not None:
#     min_days_after_change = int(args["days_after"])
# if args["min_count"] is not None:
#     min_cnt = int(args["min_count"])
# if args["min_unique"] is not None:
#     min_unique_events = int(args["min_unique"])

# Set seed for random generator
random.seed(seed)

# Load user similarity info generated by get_user_sim.py
users_data = json.load(open('user_info.txt'))
users_list = users_data['user_list']
users = users_data['user_info']

In [10]:
for user in users:
    # Replace similarity list with dictionary for easier handling
    sim_list = users[user]['similarities']
    users[user]['similarities'] = {}
    users[user]['day_list'] = sorted(users[user]['day_list'])
    index = 0
    for uid in users_list:
        users[user]['similarities'][uid] = sim_list[index]
        index += 1

# Randomly select pairs of similar users for switching
pairs = {}
change_times = {}
taken = []
choices = list(users.keys())

In [11]:
while len(pairs) / 2 < num_pairs:
    if len(choices) == 0:
        print('Could not find user pair, try to extend allowed similarity range. Aborting...')
        exit()
    # Randomly select a first user for potential switching
    first_user = random.choice(choices)
    choices.remove(first_user)
    if users[first_user]['days'] <= min_days_before_change + min_days_after_change or users[first_user]['cnt'] <= min_cnt or len(users[first_user]['actions']) <= min_unique_events:
        # First user does not fulfill requirements - skip
        continue
    # Try to find a second user to be switched with first user
    choices_inner = copy.deepcopy(choices)
    found = False
    second_user = None
    change_time_first = None
    change_time_second = None
    while found is False:
        if len(choices_inner) == 0:
            # No second user found; go back and select another first user
            break
        user_inner = random.choice(choices_inner)
        choices_inner.remove(user_inner)
        if users[user_inner]['days'] <= min_days_before_change + min_days_after_change or users[user_inner]['cnt'] <= min_cnt or len(users[user_inner]['actions']) <= min_unique_events:
            # Second user does not fulfill requirements - skip
            continue
        if users[first_user]['similarities'][user_inner] >= min_similarity and users[first_user]['similarities'][user_inner] <= max_similarity and first_user != user_inner:
            # Potential user pair for switching found; check if sufficient overlap exists for switching them
            earliest_change = max(users[first_user]['day_list'][min_days_before_change], users[user_inner]['day_list'][min_days_before_change])
            latest_change = min(users[first_user]['day_list'][-min_days_after_change], users[user_inner]['day_list'][-min_days_after_change])
            if earliest_change < latest_change:
                # Switching is possible; randomly select switching point
                ctt = random.randint(earliest_change, latest_change)
                # Find first active day of switched user behavior for ground truth
                for day_element in users[first_user]['day_list']:
                    if ctt <= day_element:
                        change_time_first = day_element
                        break
                for day_element in users[user_inner]['day_list']:
                    if ctt <= day_element:
                        change_time_second = day_element
                        break
                second_user = user_inner
                found = True
    if found is True:
        print('Changing user pair with similarity ' + str(round(users[first_user]['similarities'][second_user], 2)) + ':')
        print(' * User ' + str(first_user) + ' changed at ' + str(datetime.datetime.fromtimestamp(change_time_first, pytz.utc)) + ' (user originally carried out ' + str(users[first_user]['cnt']) + ' total events and ' + str(len(users[first_user]['actions'])) + ' unique events during ' + str(users[first_user]['days']) + ' active days).')
        print(' * User ' + str(second_user) + ' changed at ' + str(datetime.datetime.fromtimestamp(change_time_second, pytz.utc)) + ' (user originally carried out ' + str(users[second_user]['cnt']) + ' total events and ' + str(len(users[second_user]['actions'])) + ' unique events during ' + str(users[second_user]['days']) + ' active days).')
        choices.remove(second_user)
        pairs[first_user] = second_user
        pairs[second_user] = first_user
        change_times[first_user] = change_time_first
        change_times[second_user] = change_time_second


Changing user pair with similarity 0.18:
 * User intact-gray-marlin-trademarkagent changed at 2020-09-06 00:00:00+00:00 (user originally carried out 245387 total events and 15 unique events during 1026 active days).
 * User tan-crimson-mite-blindfitter changed at 2020-10-06 00:00:00+00:00 (user originally carried out 116 total events and 5 unique events during 29 active days).
Changing user pair with similarity 0.42:
 * User japanese-yellow-pike-thermalengineer changed at 2020-02-18 00:00:00+00:00 (user originally carried out 96841 total events and 23 unique events during 669 active days).
 * User visual-copper-antlion-busconductor changed at 2020-02-17 00:00:00+00:00 (user originally carried out 24089 total events and 12 unique events during 502 active days).
Changing user pair with similarity 0.22:
 * User obedient-maroon-buzzard-caremanager changed at 2020-08-31 00:00:00+00:00 (user originally carried out 34251 total events and 19 unique events during 602 active days).
 * User basic

In [12]:
# Create new file with injected anomalies
write_every = 1000000 # Write file in batches for performance
buf_cnt = 0
out_string = ""
cnt = 0
total_lines = 50522931
with open('clue.json') as f, open('clue_anomaly_train.json', 'w+') as out, open('labels.txt', 'w+') as labels:
    for line in f:
        cnt += 1
        buf_cnt += 1
        if int(cnt % (total_lines / 20)) == 0: 
            print(str(int(cnt*100/total_lines)) + '%', end=' ', flush=True)
        j = json.loads(line)
        uid = j['uid']
        # Check if uid is in one of the selected user pairs and switch accordingly
        if uid in change_times:
            ts = parser.isoparse(j['time'])
            if ts.timestamp() >= change_times[uid]:
                j['uid'] = pairs[uid]
                if 'user' in j['params'] and j['params']['user'] == uid:
                    # Also switch parameter user when it occurs like uid
                    j['params']['user'] = pairs[uid]
        # Write updated events to file
        out_string += json.dumps(j) + '\n'
        if buf_cnt >= write_every:
            buf_cnt = 0
            out.write(out_string)
            out_string = ""
    out.write(out_string) # Write remaining output after final loop
    for uid, time in change_times.items():
        labels.write(str(pairs[uid]) + ',' + str(time) + '\n') # Need to use pairs[uid] instead of uid since we want to detect the first activity of the replacement-user
print('')

5% 10% 15% 20% 25% 30% 35% 40% 45% 50% 55% 60% 65% 70% 75% 80% 85% 90% 95% 100% 


In [13]:
!cat labels.txt

tan-crimson-mite-blindfitter,1599350400.0
intact-gray-marlin-trademarkagent,1601942400.0
visual-copper-antlion-busconductor,1581984000.0
japanese-yellow-pike-thermalengineer,1581897600.0
basic-pink-fox-sharedealer,1598832000.0
obedient-maroon-buzzard-caremanager,1622160000.0
basic-blue-gull-physiotherapist,1530835200.0
repulsive-violet-canidae-warehousewoman,1530835200.0
uncertain-lavender-barracuda-ornithologist,1542844800.0
green-green-piranha-nurserynurse,1554336000.0
steady-jade-leech-payrollsupervisor,1525219200.0
isolated-pink-dingo-nightwatchman,1524700800.0
giant-azure-coyote-aeronauticalengineer,1580688000.0
victorious-gray-clam-taxadvisor,1580688000.0
nice-orange-centipede-taxidriver,1503792000.0
changing-yellow-landfowl-mastermariner,1503792000.0
integral-turquoise-narwhal-rugmaker,1529625600.0
precise-coffee-dog-locksmith,1530057600.0
delicious-ivory-crocodile-textileengineer,1570060800.0
defiant-yellow-bobolink-panelbeater,1569888000.0
unemployed-harlequin-swordtail-optome

# Detect anomalies

user@ubuntu:~/clue-lds$ python3 detect.py -t 0.6
Ground truth:
 * shared-fuchsia-cardinal-buildingadvisor switched at 2022-09-28 00:00:00+00:00
 * competent-aqua-hare-buildinginspector switched at 2022-09-28 00:00:00+00:00
 * japanese-yellow-pike-thermalengineer switched at 2018-06-10 00:00:00+00:00
 * dull-amethyst-buzzard-ledgerclerk switched at 2018-06-11 00:00:00+00:00
...


 5389 users with 83147 days considered, including days spent on training and incomplete days.
Results with threshold = 0.6:
  Total = 72469
  Train = 5289
  Detect = 72469
  Detected users = ['shared-fuchsia-cardinal-buildingadvisor', 'competent-aqua-hare-buildinginspector', 'japanese-yellow-pike-thermalengineer', 'dull-amethyst-buzzard-ledgerclerk', 'graceful-olive-spoonbill-careersofficer', 'high-chocolate-emu-liftengineer', 'careful-coffee-fowl-trafficwarden', 'southern-brown-gerbil-medicalsecretary', 'ethnic-lavender-gerbil-gamingclubmanager', 'modern-coral-crocodile-lampshademaker', 'extraordinary-plum-clownfish-sawmiller', 'hurt-aqua-roundworm-fuelmerchant', 'proud-copper-marmoset-accountsclerk']
  Missed users = ['chosen-bronze-egret-ticketagent', 'apparent-apricot-lamprey-artexer', 'horrible-moccasin-mole-licensing', 'famous-lavender-sailfish-partitionerector', 'meaningful-blue-viper-tankerdriver', 'labour-crimson-donkey-golfcaddy', 'ambitious-gold-bonobo-repairman']
  TP_adj = 13
  TP = 13
  FP = 3337
  TN = 69112
  FN_adj = 7
  FN = 7
  TPR_adj = Rec_adj = 0.65
  TPR = Rec = 0.65
  FPR = 0.04605998702535577
  TNR = 0.9539400129746443
  Prec = 0.003880597014925373
  F1 = 0.00771513353115727
  ACC = 0.9538561315872994
  R = 0.06801872476143933
  Runtime = 655.9123327732086

In [14]:
import json
from dateutil import parser
import datetime
import pytz
import math
import sys
import argparse
import time

threshold = 0.6
anom_free_days = 1
mode = 1
queue = 30 # -1 for unlimited
update = False
debug_out = True
fifo = False # When false matching vectors are moved to the front of the queue to keep them from being deleted

# argparser = argparse.ArgumentParser(description='Detection.')
# argparser.add_argument('-t','--thresh', help='Similarity threshold.', required=False)
# argparser.add_argument('-r','--retrain', help='Retrain length (days).', required=False)
# argparser.add_argument('-m','--mode', help='1 .. default, 2 .. idf, 3 .. norm', required=False)
# argparser.add_argument('-q','--queue', help='Queue size (-1 for unlimited).', required=False)
# argparser.add_argument('-u','--update', help='Update model also during detection.', required=False, action='store_true')
# argparser.add_argument('-f','--fifo', help='Do not move matching vectors to front of queue.', required=False, action='store_true')
# argparser.add_argument('-d','--debug', help='Output debug information.', required=False, action='store_true')

# args = vars(argparser.parse_args())
# if args["thresh"] is not None:
#     threshold = float(args["thresh"])
# if args["retrain"] is not None:
#     anom_free_days = int(args["retrain"])
# if args["mode"] is not None:
#     mode = int(args["mode"]) # 1 .. normal, 2 .. idf, 3 .. norm
# if args["queue"] is not None:
#     queue = int(args["queue"])
# update = args["update"]
# debug_out = args["debug"]
# fifo = args["fifo"]

# Read in ground truth (switched uid and corresponding timestamps)
anomalous_users = {}
with open('labels.txt') as f:
    print('Ground truth: ')
    for line in f:
        parts = line.split(',')
        ts = datetime.datetime.fromtimestamp(int(float(parts[1])), tz=pytz.UTC)
        anomalous_users[parts[0]] = datetime.datetime(ts.year, ts.month, ts.day, 0, 0, 0, 0, pytz.UTC) # Omit time info since we count detected days
        print(' * ' + str(parts[0]) + ' switched at ' + str(ts))

total_days = {}
wait_days = {}
cnt = 0
detected_dist = {}
last_anom_dist = {}
freq_day = {}
last_active_day = {}
dists = {}
debug = {}
idf = {}
only_anomalous_users = False # Skip normal users that are not in the ground truth (mainly for debugging/testing)
total_lines = 50522931
start_time = time.time()
with open('clue_anomaly.json') as f:
    for line in f:
        cnt += 1
        if int(cnt % (total_lines / 20)) == 0:
            print(str(int(cnt*100/total_lines)) + '%', end=' ', flush=True)
        j = json.loads(line)
        uid = j['uid']
        if only_anomalous_users and uid not in anomalous_users:
            # Only consider anomalous users and skip normal users for analysis
            continue
        action = j['type']
        # Count by how many users each event type is used for idf-weighting if mode is set to idf
        if action not in idf:
            idf[action] = set([uid])
        else:
            idf[action].add(uid)
        ts = parser.isoparse(j['time'])
        currentday = datetime.datetime(ts.year, ts.month, ts.day, 0, 0, 0, 0, pytz.UTC) # Omit hour, minute, and second
        # Store all days where each user is active
        if uid not in total_days:
            total_days[uid] = set([currentday])
        else:
            total_days[uid].add(currentday)
        if uid not in last_active_day:
            # First appearance of uid - initialize user information
            freq_day[uid] = {}
            freq_day[uid][action] = 1
            wait_days[uid] = anom_free_days
            debug[uid] = []
            last_active_day[uid] = currentday
            dists[uid] = []
        else:
            if last_active_day[uid] != currentday:
                # Start of a new day - check count vector of previous day
                min_dist = None
                min_known = None
                min_limit = None
                duplicate_check = []
                for known in dists[uid]:
                    # Check all count vectors present in model
                    if known in duplicate_check:
                        # Identical count vector already checked, no need to check again
                        continue
                    duplicate_check.append(known)
                    # Initialize distance measures
                    manh = 0
                    limit = 0
                    for action_element in set(list(known.keys()) + list(freq_day[uid].keys())):
                        # Go through all event types that are either in the model count vector or checked count vector
                        idf_fact = 1
                        if mode == 2:
                            # Weigh event types lower when many users use them
                            idf_fact = math.log10((1 + len(last_active_day)) / len(idf[action_element]))
                        norm_sum_known = 1
                        norm_sum_freq = 1
                        if mode == 3:
                            # Normalize event count vectors so that only relative frequencies matter
                            norm_sum_known = sum(known.values())
                            norm_sum_freq = sum(freq_day[uid].values())
                        # Increase distance measures depending on how often each event type occurs in model and checked count vector
                        if action_element not in known:
                            # Event type only in checked count vector - increase distance by count value
                            manh += freq_day[uid][action_element] * idf_fact / norm_sum_freq
                            limit += freq_day[uid][action_element] * idf_fact / norm_sum_freq
                        elif action_element not in freq_day[uid]:
                            # Event type only in model count vector - increase distance by count value
                            manh += known[action_element] * idf_fact / norm_sum_known
                            limit += known[action_element] * idf_fact / norm_sum_known
                        else:
                            # Event type in both model and checked count vector - increase distance by the absolute difference (0 for perfect match)
                            manh += abs(freq_day[uid][action_element] * idf_fact / norm_sum_freq - known[action_element] * idf_fact / norm_sum_known)
                            # Increase limit by maximum of both count values as it is the upper limit for the distance
                            limit += max(freq_day[uid][action_element] * idf_fact / norm_sum_freq, known[action_element] * idf_fact / norm_sum_known)
                    if min_dist is None:
                        # Initialize minimum distance
                        min_dist = manh / limit # anomaly score (how close is distance to upper limit)
                        min_known = known # most similar vector from model
                        #limit = sum(list(map(add, norm, known))) # max(sum(norm), sum(known_norm)) / 2
                        min_limit = limit
                    else:
                        # Check for smaller distance
                        if manh / limit < min_dist:
                            min_dist = manh / limit
                            min_known = known
                            min_limit = limit
                    if min_dist == 0:
                        # Perfect match, no need to go over all other elements
                        break
                # Get date of previous (and currently analyzed) day
                last_active_day_date = datetime.datetime(last_active_day[uid].year, last_active_day[uid].month, last_active_day[uid].day, 0, 0, 0, 0, pytz.UTC)
                # Each user on each day where they are active is subject of detection
                sample = (uid, last_active_day_date)
                if sample not in detected_dist:
                    detected_dist[sample] = []
                if wait_days[uid] <= 0:
                    # Detection is currently ongoing as sufficient days have passed since last anomaly
                    detected_dist[sample].append("detection")
                else:
                    # Re-training is currently ongoing
                    detected_dist[sample].append("training")
                if min_dist is not None and min_dist > threshold:
                    # Model is empty or count vector is considered anomalous
                    detected_dist[sample].append("anomalous")
                    # Restart re-training
                    wait_days[uid] = anom_free_days
                else:
                    # Count vector is considered normal
                    detected_dist[sample].append("normal")
                    # Reduce days left for re-training
                    wait_days[uid] -= 1
                    if fifo is False and min_known is not None:
                        # Move matching min_known vector to end of list to save it from aging out
                        dists[uid].remove(min_known)
                        dists[uid].append(min_known)
                if update is True or wait_days[uid] >= 0:
                    # Add new count vector to model when model is updated also for normal data or currently re-training
                    dists[uid].append(freq_day[uid])
                if queue != -1 and len(dists[uid]) > queue:
                    # Remove oldest count vector from queue in model
                    dists[uid] = dists[uid][1:]
                if debug_out is True:
                    debug[uid].append((datetime.datetime(last_active_day[uid].year, last_active_day[uid].month, last_active_day[uid].day, 0, 0, 0, 0, pytz.UTC), min_dist, min_limit, detected_dist[sample], min_known, freq_day[uid], dists[uid]))
                last_active_day[uid] = currentday
                freq_day[uid] = {}
            if action in freq_day[uid]:
                freq_day[uid][action] += 1
            else:
                freq_day[uid][action] = 1

if debug_out is True:
    # Print anomaly scores for each user and each day
    with open('debug.txt', 'w+') as out:
        for uid, lst in debug.items():
            if len(lst) > 15: # Skip users with few active days
                out.write('\n' + uid + '\n')
                for elem in lst:
                    string = ""
                    if uid in anomalous_users and anomalous_users[uid].timestamp() <= elem[0].timestamp():
                        string += " Changed user!"
                    if elem[1] is not None and elem[2] is not None and elem[1] > elem[2]:
                        string += ' Detected!'
                    if elem[1] is not None and elem[2] is not None:
                        out.write(str(elem[0]) + ': ' + str(elem[1] * elem[2]) + '/' + str(elem[2]) + ' #' + str(round(elem[1], 2)) + string + ' ' + str(elem[3]) + '\n')
                    if uid in anomalous_users and abs(anomalous_users[uid].timestamp() - elem[0].timestamp()) < 60*60*24*60 and elem[4] is not None and elem[5] is not None:
                        # Also print count vectors for days close to switching anomalous users
                        for model_vec in elem[6]:
                            out.write('Model vec.: ' + str(dict(sorted(model_vec.items()))) + '\n')
                        out.write('Best match: ' + str(dict(sorted(elem[4].items()))) + '\n')
                        out.write('Count vec.: ' + str(dict(sorted(elem[5].items()))) + '\n')

def get_eval_results(d):
    # Initialize metrics
    tp = 0
    tp_adjusted = 0
    fp = 0
    tn = 0
    fn = 0
    fn_adjusted = 0
    tp_user = {}
    tp_adjusted_user = {}
    fn_user = {}
    for uid in anomalous_users:
        # Count detections separately for each anomalous user
        tp_user[uid] = 0
        tp_adjusted_user[uid] = 0
        fn_user[uid] = 0
    sum_training = 0
    sum_detection = 0
    for tup, detected in d.items():
        if "training" in detected:
            # Count all days for all users spent for (re-)training
            sum_training += 1
        elif "detection" in detected:
            # Count all days for all users spent for detecting
            sum_detection += 1
        uid = tup[0]
        day = tup[1]
        # Check if detected user is in ground truth, the timestamp is after the switching point, and that only first day after switch is considered
        if uid in anomalous_users and day.timestamp() == anomalous_users[uid].timestamp():
            if detected == ["detection", "anomalous"]:
                # Normal correct detection during detecting phase
                tp += 1
                tp_user[uid] += 1
                tp_adjusted += 1
                tp_adjusted_user[uid] += 1
            elif detected == ["detection", "normal"] or detected == ["training", "normal"]:
                # Missed anomalous user (classified as normal) either during training or detection phase
                fn_adjusted += 1
                fn_user[uid] += 1
                fn += 1
            elif detected == ["training", "anomalous"]:
                # Correct detection during training phase counted by adjusted score
                tp_adjusted += 1
                tp_adjusted_user[uid] += 1
                fn += 1
        else:
            # Note that instances are omitted in the training phase
            if detected == ["detection", "anomalous"]:
                # Incorrect detection of normal behavior during detection phase
                fp += 1
            elif detected == ["detection", "normal"]:
                # Correctly non-detected instance during detection phase
                tn += 1
    # Print all metrics to console
    print('  Total = ' + str(tp + tn + fp + fn))
    print('  Train = ' + str(sum_training))
    print('  Detect = ' + str(sum_detection))
    users_detected = []
    users_undetected = []
    for uid in anomalous_users:
        if tp_adjusted_user[uid] == 1 and fn_user[uid] == 0:
            users_detected.append(uid)
        elif tp_adjusted_user[uid] == 0 and fn_user[uid] == 1:
            users_undetected.append(uid)
        else:
            print('Eval Error: ' + str(uid) + ' -> TP = ' + str(tp_adjusted_user[uid]) + ' and FN = ' + str(fn_user[uid]))
    print('  Detected users = ' + str(users_detected))
    print('  Missed users = ' + str(users_undetected))
    print('  TP_adj = ' + str(tp_adjusted))
    print('  TP = ' + str(tp))
    print('  FP = ' + str(fp))
    print('  TN = ' + str(tn))
    print('  FN_adj = ' + str(fn_adjusted))
    print('  FN = ' + str(fn))
    tpr_adjusted = "NaN"
    if tp_adjusted + fn > 0:
        tpr_adjusted = tp_adjusted / (tp_adjusted + fn_adjusted)
    print('  TPR_adj = Rec_adj = ' + str(tpr_adjusted))
    tpr = "NaN"
    if tp + fn > 0:
        tpr = tp / (tp + fn)
    print('  TPR = Rec = ' + str(tpr))
    fpr = "NaN"
    if fp + tn > 0:
        fpr = fp / (fp + tn)
    print('  FPR = ' + str(fpr))
    tnr = "NaN"
    if tn + fp > 0:
        tnr = tn / (tn + fp)
    print('  TNR = ' + str(tnr))
    prec = "NaN"
    if tp_adjusted + fp > 0:
        prec = tp_adjusted / (tp_adjusted + fp)
    print('  Prec = ' + str(prec))
    fone = "NaN"
    if tp_adjusted + 0.5 * (fp + fn_adjusted) > 0:
        fone = tp_adjusted / (tp_adjusted + 0.5 * (fp + fn_adjusted))
    print('  F1 = ' + str(fone))
    acc = "NaN"
    if tp_adjusted + tn + fp + fn_adjusted > 0:
        acc = (tp_adjusted + tn) / (tp_adjusted + tn + fp + fn_adjusted)
    print('  ACC = ' + str(acc))
    print('  R = ' + str(sum_training / (sum_training + sum_detection)))
    runtime = time.time()-start_time
    print('  Runtime = ' + str(runtime))
    # The following output is used to create a CSV of results
    print('thresh,retrain,mode,queue,update,total,train,detect,tp_adj,tp,fp,tn,fn_adj,fn,tpr_adj,tpr,fpr,tnr,p,f1,acc,time')
    print(str(threshold) + ',' + str(anom_free_days) + ',' + str(mode) + ',' + str(queue) + ',' + str(update) + ',' + str(tp + tn + fp + fn) + ',' + str(sum_training) + ',' + str(sum_detection) + ',' + str(tp_adjusted) + ',' + str(tp) + ',' + str(fp) + ',' + str(tn) + ',' + str(fn_adjusted) + ',' + str(fn) + ',' + str(tpr_adjusted) + ',' + str(tpr) + ',' + str(fpr) + ',' + str(tnr) + ',' + str(prec) + ',' + str(fone) + ',' + str(acc) + ',' + str(runtime))
    print('')

Ground truth: 
 * tan-crimson-mite-blindfitter switched at 2020-09-06 00:00:00+00:00
 * intact-gray-marlin-trademarkagent switched at 2020-10-06 00:00:00+00:00
 * visual-copper-antlion-busconductor switched at 2020-02-18 00:00:00+00:00
 * japanese-yellow-pike-thermalengineer switched at 2020-02-17 00:00:00+00:00
 * basic-pink-fox-sharedealer switched at 2020-08-31 00:00:00+00:00
 * obedient-maroon-buzzard-caremanager switched at 2021-05-28 00:00:00+00:00
 * basic-blue-gull-physiotherapist switched at 2018-07-06 00:00:00+00:00
 * repulsive-violet-canidae-warehousewoman switched at 2018-07-06 00:00:00+00:00
 * uncertain-lavender-barracuda-ornithologist switched at 2018-11-22 00:00:00+00:00
 * green-green-piranha-nurserynurse switched at 2019-04-04 00:00:00+00:00
 * steady-jade-leech-payrollsupervisor switched at 2018-05-02 00:00:00+00:00
 * isolated-pink-dingo-nightwatchman switched at 2018-04-26 00:00:00+00:00
 * giant-azure-coyote-aeronauticalengineer switched at 2020-02-03 00:00:00+00

In [15]:
sum_total_days = 0
for uid, total_day_count in total_days.items():
    sum_total_days += len(total_day_count)
print('\n ' + str(len(total_days)) + ' users with ' + str(sum_total_days) + ' days considered, including days spent on training and incomplete days.')

print('Results with threshold = ' + str(threshold) + ':')
get_eval_results(detected_dist)


 5389 users with 83147 days considered, including days spent on training and incomplete days.
Results with threshold = 0.6:
  Total = 72474
  Train = 5289
  Detect = 72469
Eval Error: tan-crimson-mite-blindfitter -> TP = 0 and FN = 0
Eval Error: japanese-yellow-pike-thermalengineer -> TP = 0 and FN = 0
Eval Error: basic-pink-fox-sharedealer -> TP = 0 and FN = 0
Eval Error: uncertain-lavender-barracuda-ornithologist -> TP = 0 and FN = 0
Eval Error: green-green-piranha-nurserynurse -> TP = 0 and FN = 0
Eval Error: isolated-pink-dingo-nightwatchman -> TP = 0 and FN = 0
Eval Error: integral-turquoise-narwhal-rugmaker -> TP = 0 and FN = 0
Eval Error: delicious-ivory-crocodile-textileengineer -> TP = 0 and FN = 0
Eval Error: defiant-yellow-bobolink-panelbeater -> TP = 0 and FN = 0
Eval Error: unemployed-harlequin-swordtail-optometrist -> TP = 0 and FN = 0
Eval Error: horrible-moccasin-mole-licensing -> TP = 0 and FN = 0
Eval Error: inherent-blue-leopon-glassworker -> TP = 0 and FN = 0
Eval 

In [16]:
total_day_count

{datetime.datetime(2022, 9, 29, 0, 0, tzinfo=<UTC>)}

In [17]:
total_days['yellow-chocolate-carp-busdriver']

{datetime.datetime(2017, 7, 7, 0, 0, tzinfo=<UTC>),
 datetime.datetime(2017, 7, 10, 0, 0, tzinfo=<UTC>),
 datetime.datetime(2017, 7, 11, 0, 0, tzinfo=<UTC>),
 datetime.datetime(2017, 7, 12, 0, 0, tzinfo=<UTC>),
 datetime.datetime(2017, 7, 13, 0, 0, tzinfo=<UTC>),
 datetime.datetime(2017, 7, 17, 0, 0, tzinfo=<UTC>),
 datetime.datetime(2017, 7, 18, 0, 0, tzinfo=<UTC>),
 datetime.datetime(2017, 7, 19, 0, 0, tzinfo=<UTC>),
 datetime.datetime(2017, 7, 20, 0, 0, tzinfo=<UTC>),
 datetime.datetime(2017, 7, 21, 0, 0, tzinfo=<UTC>),
 datetime.datetime(2017, 7, 24, 0, 0, tzinfo=<UTC>),
 datetime.datetime(2017, 7, 25, 0, 0, tzinfo=<UTC>),
 datetime.datetime(2017, 7, 27, 0, 0, tzinfo=<UTC>),
 datetime.datetime(2017, 7, 28, 0, 0, tzinfo=<UTC>),
 datetime.datetime(2017, 7, 30, 0, 0, tzinfo=<UTC>),
 datetime.datetime(2017, 7, 31, 0, 0, tzinfo=<UTC>),
 datetime.datetime(2017, 8, 6, 0, 0, tzinfo=<UTC>),
 datetime.datetime(2017, 8, 7, 0, 0, tzinfo=<UTC>),
 datetime.datetime(2017, 8, 8, 0, 0, tzinfo=<UTC>

# Debug

In [18]:
!head debug.txt -n 100


yellow-chocolate-carp-busdriver
2017-07-10 00:00:00+00:00: 57.0/63.0 #0.9 ['detection', 'anomalous']
2017-07-11 00:00:00+00:00: 2.0/6.0 #0.33 ['training', 'normal']
2017-07-12 00:00:00+00:00: 76.0/99.0 #0.77 ['detection', 'anomalous']
2017-07-13 00:00:00+00:00: 7.0/9.0 #0.78 ['training', 'anomalous']
2017-07-17 00:00:00+00:00: 6.0/9.0 #0.67 ['training', 'anomalous']
2017-07-18 00:00:00+00:00: 4.0/5.0 #0.8 ['training', 'anomalous']
2017-07-19 00:00:00+00:00: 2.0/5.0 #0.4 ['training', 'normal']
2017-07-20 00:00:00+00:00: 53.0/87.0 #0.61 ['detection', 'anomalous']
2017-07-21 00:00:00+00:00: 5.0/6.0 #0.83 ['training', 'anomalous']
2017-07-24 00:00:00+00:00: 7.0/12.0 #0.58 ['training', 'normal']
2017-07-25 00:00:00+00:00: 4.0/8.0 #0.5 ['detection', 'normal']
2017-07-27 00:00:00+00:00: 6.0/12.0 #0.5 ['detection', 'normal']
2017-07-28 00:00:00+00:00: 43.0/68.0 #0.63 ['detection', 'anomalous']
2017-07-30 00:00:00+00:00: 149.0/170.0 #0.88 ['training', 'anomalous']
2017-07-31 00:00:00+00:00: 97

# Only anomalies

In [19]:
import json
from dateutil import parser
import datetime
import pytz
import math
import sys
import argparse
import time

threshold = 0.6
anom_free_days = 1
mode = 1
queue = 30 # -1 for unlimited
update = False
debug_out = False
fifo = False # When false matching vectors are moved to the front of the queue to keep them from being deleted


# Read in ground truth (switched uid and corresponding timestamps)
anomalous_users = {}
with open('labels.txt') as f:
    print('Ground truth: ')
    for line in f:
        parts = line.split(',')
        ts = datetime.datetime.fromtimestamp(int(float(parts[1])), tz=pytz.UTC)
        anomalous_users[parts[0]] = datetime.datetime(ts.year, ts.month, ts.day, 0, 0, 0, 0, pytz.UTC) # Omit time info since we count detected days
        print(' * ' + str(parts[0]) + ' switched at ' + str(ts))

total_days = {}
wait_days = {}
cnt = 0
detected_dist = {}
last_anom_dist = {}
freq_day = {}
last_active_day = {}
dists = {}
debug = {}
idf = {}
only_anomalous_users = True # Skip normal users that are not in the ground truth (mainly for debugging/testing)
total_lines = 50522931
start_time = time.time()
with open('clue_anomaly.json') as f:
    for line in f:
        cnt += 1
        if int(cnt % (total_lines / 20)) == 0:
            print(str(int(cnt*100/total_lines)) + '%', end=' ', flush=True)
        j = json.loads(line)
        uid = j['uid']
        if only_anomalous_users and uid not in anomalous_users:
            # Only consider anomalous users and skip normal users for analysis
            continue
        action = j['type']
        # Count by how many users each event type is used for idf-weighting if mode is set to idf
        if action not in idf:
            idf[action] = set([uid])
        else:
            idf[action].add(uid)
        ts = parser.isoparse(j['time'])
        currentday = datetime.datetime(ts.year, ts.month, ts.day, 0, 0, 0, 0, pytz.UTC) # Omit hour, minute, and second
        # Store all days where each user is active
        if uid not in total_days:
            total_days[uid] = set([currentday])
        else:
            total_days[uid].add(currentday)
        if uid not in last_active_day:
            # First appearance of uid - initialize user information
            freq_day[uid] = {}
            freq_day[uid][action] = 1
            wait_days[uid] = anom_free_days
            debug[uid] = []
            last_active_day[uid] = currentday
            dists[uid] = []
        else:
            if last_active_day[uid] != currentday:
                # Start of a new day - check count vector of previous day
                min_dist = None
                min_known = None
                min_limit = None
                duplicate_check = []
                for known in dists[uid]:
                    # Check all count vectors present in model
                    if known in duplicate_check:
                        # Identical count vector already checked, no need to check again
                        continue
                    duplicate_check.append(known)
                    # Initialize distance measures
                    manh = 0
                    limit = 0
                    for action_element in set(list(known.keys()) + list(freq_day[uid].keys())):
                        # Go through all event types that are either in the model count vector or checked count vector
                        idf_fact = 1
                        if mode == 2:
                            # Weigh event types lower when many users use them
                            idf_fact = math.log10((1 + len(last_active_day)) / len(idf[action_element]))
                        norm_sum_known = 1
                        norm_sum_freq = 1
                        if mode == 3:
                            # Normalize event count vectors so that only relative frequencies matter
                            norm_sum_known = sum(known.values())
                            norm_sum_freq = sum(freq_day[uid].values())
                        # Increase distance measures depending on how often each event type occurs in model and checked count vector
                        if action_element not in known:
                            # Event type only in checked count vector - increase distance by count value
                            manh += freq_day[uid][action_element] * idf_fact / norm_sum_freq
                            limit += freq_day[uid][action_element] * idf_fact / norm_sum_freq
                        elif action_element not in freq_day[uid]:
                            # Event type only in model count vector - increase distance by count value
                            manh += known[action_element] * idf_fact / norm_sum_known
                            limit += known[action_element] * idf_fact / norm_sum_known
                        else:
                            # Event type in both model and checked count vector - increase distance by the absolute difference (0 for perfect match)
                            manh += abs(freq_day[uid][action_element] * idf_fact / norm_sum_freq - known[action_element] * idf_fact / norm_sum_known)
                            # Increase limit by maximum of both count values as it is the upper limit for the distance
                            limit += max(freq_day[uid][action_element] * idf_fact / norm_sum_freq, known[action_element] * idf_fact / norm_sum_known)
                    if min_dist is None:
                        # Initialize minimum distance
                        min_dist = manh / limit # anomaly score (how close is distance to upper limit)
                        min_known = known # most similar vector from model
                        #limit = sum(list(map(add, norm, known))) # max(sum(norm), sum(known_norm)) / 2
                        min_limit = limit
                    else:
                        # Check for smaller distance
                        if manh / limit < min_dist:
                            min_dist = manh / limit
                            min_known = known
                            min_limit = limit
                    if min_dist == 0:
                        # Perfect match, no need to go over all other elements
                        break
                # Get date of previous (and currently analyzed) day
                last_active_day_date = datetime.datetime(last_active_day[uid].year, last_active_day[uid].month, last_active_day[uid].day, 0, 0, 0, 0, pytz.UTC)
                # Each user on each day where they are active is subject of detection
                sample = (uid, last_active_day_date)
                if sample not in detected_dist:
                    detected_dist[sample] = []
                if wait_days[uid] <= 0:
                    # Detection is currently ongoing as sufficient days have passed since last anomaly
                    detected_dist[sample].append("detection")
                else:
                    # Re-training is currently ongoing
                    detected_dist[sample].append("training")
                if min_dist is not None and min_dist > threshold:
                    # Model is empty or count vector is considered anomalous
                    detected_dist[sample].append("anomalous")
                    # Restart re-training
                    wait_days[uid] = anom_free_days
                else:
                    # Count vector is considered normal
                    detected_dist[sample].append("normal")
                    # Reduce days left for re-training
                    wait_days[uid] -= 1
                    if fifo is False and min_known is not None:
                        # Move matching min_known vector to end of list to save it from aging out
                        dists[uid].remove(min_known)
                        dists[uid].append(min_known)
                if update is True or wait_days[uid] >= 0:
                    # Add new count vector to model when model is updated also for normal data or currently re-training
                    dists[uid].append(freq_day[uid])
                if queue != -1 and len(dists[uid]) > queue:
                    # Remove oldest count vector from queue in model
                    dists[uid] = dists[uid][1:]
                if debug_out is True:
                    debug[uid].append((datetime.datetime(last_active_day[uid].year, last_active_day[uid].month, last_active_day[uid].day, 0, 0, 0, 0, pytz.UTC), min_dist, min_limit, detected_dist[sample], min_known, freq_day[uid], dists[uid]))
                last_active_day[uid] = currentday
                freq_day[uid] = {}
            if action in freq_day[uid]:
                freq_day[uid][action] += 1
            else:
                freq_day[uid][action] = 1

if debug_out is True:
    # Print anomaly scores for each user and each day
    with open('debug.txt', 'w+') as out:
        for uid, lst in debug.items():
            if len(lst) > 15: # Skip users with few active days
                out.write('\n' + uid + '\n')
                for elem in lst:
                    string = ""
                    if uid in anomalous_users and anomalous_users[uid].timestamp() <= elem[0].timestamp():
                        string += " Changed user!"
                    if elem[1] is not None and elem[2] is not None and elem[1] > elem[2]:
                        string += ' Detected!'
                    if elem[1] is not None and elem[2] is not None:
                        out.write(str(elem[0]) + ': ' + str(elem[1] * elem[2]) + '/' + str(elem[2]) + ' #' + str(round(elem[1], 2)) + string + ' ' + str(elem[3]) + '\n')
                    if uid in anomalous_users and abs(anomalous_users[uid].timestamp() - elem[0].timestamp()) < 60*60*24*60 and elem[4] is not None and elem[5] is not None:
                        # Also print count vectors for days close to switching anomalous users
                        for model_vec in elem[6]:
                            out.write('Model vec.: ' + str(dict(sorted(model_vec.items()))) + '\n')
                        out.write('Best match: ' + str(dict(sorted(elem[4].items()))) + '\n')
                        out.write('Count vec.: ' + str(dict(sorted(elem[5].items()))) + '\n')

def get_eval_results(d):
    # Initialize metrics
    tp = 0
    tp_adjusted = 0
    fp = 0
    tn = 0
    fn = 0
    fn_adjusted = 0
    tp_user = {}
    tp_adjusted_user = {}
    fn_user = {}
    for uid in anomalous_users:
        # Count detections separately for each anomalous user
        tp_user[uid] = 0
        tp_adjusted_user[uid] = 0
        fn_user[uid] = 0
    sum_training = 0
    sum_detection = 0
    for tup, detected in d.items():
        if "training" in detected:
            # Count all days for all users spent for (re-)training
            sum_training += 1
        elif "detection" in detected:
            # Count all days for all users spent for detecting
            sum_detection += 1
        uid = tup[0]
        day = tup[1]
        # Check if detected user is in ground truth, the timestamp is after the switching point, and that only first day after switch is considered
        if uid in anomalous_users and day.timestamp() == anomalous_users[uid].timestamp():
            if detected == ["detection", "anomalous"]:
                # Normal correct detection during detecting phase
                tp += 1
                tp_user[uid] += 1
                tp_adjusted += 1
                tp_adjusted_user[uid] += 1
            elif detected == ["detection", "normal"] or detected == ["training", "normal"]:
                # Missed anomalous user (classified as normal) either during training or detection phase
                fn_adjusted += 1
                fn_user[uid] += 1
                fn += 1
            elif detected == ["training", "anomalous"]:
                # Correct detection during training phase counted by adjusted score
                tp_adjusted += 1
                tp_adjusted_user[uid] += 1
                fn += 1
        else:
            # Note that instances are omitted in the training phase
            if detected == ["detection", "anomalous"]:
                # Incorrect detection of normal behavior during detection phase
                fp += 1
            elif detected == ["detection", "normal"]:
                # Correctly non-detected instance during detection phase
                tn += 1
    # Print all metrics to console
    print('  Total = ' + str(tp + tn + fp + fn))
    print('  Train = ' + str(sum_training))
    print('  Detect = ' + str(sum_detection))
    users_detected = []
    users_undetected = []
    for uid in anomalous_users:
        if tp_adjusted_user[uid] == 1 and fn_user[uid] == 0:
            users_detected.append(uid)
        elif tp_adjusted_user[uid] == 0 and fn_user[uid] == 1:
            users_undetected.append(uid)
        else:
            print('Eval Error: ' + str(uid) + ' -> TP = ' + str(tp_adjusted_user[uid]) + ' and FN = ' + str(fn_user[uid]))
    print('  Detected users = ' + str(users_detected))
    print('  Missed users = ' + str(users_undetected))
    print('  TP_adj = ' + str(tp_adjusted))
    print('  TP = ' + str(tp))
    print('  FP = ' + str(fp))
    print('  TN = ' + str(tn))
    print('  FN_adj = ' + str(fn_adjusted))
    print('  FN = ' + str(fn))
    tpr_adjusted = "NaN"
    if tp_adjusted + fn > 0:
        tpr_adjusted = tp_adjusted / (tp_adjusted + fn_adjusted)
    print('  TPR_adj = Rec_adj = ' + str(tpr_adjusted))
    tpr = "NaN"
    if tp + fn > 0:
        tpr = tp / (tp + fn)
    print('  TPR = Rec = ' + str(tpr))
    fpr = "NaN"
    if fp + tn > 0:
        fpr = fp / (fp + tn)
    print('  FPR = ' + str(fpr))
    tnr = "NaN"
    if tn + fp > 0:
        tnr = tn / (tn + fp)
    print('  TNR = ' + str(tnr))
    prec = "NaN"
    if tp_adjusted + fp > 0:
        prec = tp_adjusted / (tp_adjusted + fp)
    print('  Prec = ' + str(prec))
    fone = "NaN"
    if tp_adjusted + 0.5 * (fp + fn_adjusted) > 0:
        fone = tp_adjusted / (tp_adjusted + 0.5 * (fp + fn_adjusted))
    print('  F1 = ' + str(fone))
    acc = "NaN"
    if tp_adjusted + tn + fp + fn_adjusted > 0:
        acc = (tp_adjusted + tn) / (tp_adjusted + tn + fp + fn_adjusted)
    print('  ACC = ' + str(acc))
    print('  R = ' + str(sum_training / (sum_training + sum_detection)))
    runtime = time.time()-start_time
    print('  Runtime = ' + str(runtime))
    # The following output is used to create a CSV of results
    #print('thresh,retrain,mode,queue,update,total,train,detect,tp_adj,tp,fp,tn,fn_adj,fn,tpr_adj,tpr,fpr,tnr,p,f1,acc,time')
    #print(str(threshold) + ',' + str(anom_free_days) + ',' + str(mode) + ',' + str(queue) + ',' + str(update) + ',' + str(tp + tn + fp + fn) + ',' + str(sum_training) + ',' + str(sum_detection) + ',' + str(tp_adjusted) + ',' + str(tp) + ',' + str(fp) + ',' + str(tn) + ',' + str(fn_adjusted) + ',' + str(fn) + ',' + str(tpr_adjusted) + ',' + str(tpr) + ',' + str(fpr) + ',' + str(tnr) + ',' + str(prec) + ',' + str(fone) + ',' + str(acc) + ',' + str(runtime))
    print('')

sum_total_days = 0
for uid, total_day_count in total_days.items():
    sum_total_days += len(total_day_count)
print('\n ' + str(len(total_days)) + ' users with ' + str(sum_total_days) + ' days considered, including days spent on training and incomplete days.')

print('Results with threshold = ' + str(threshold) + ':')
get_eval_results(detected_dist)

Ground truth: 
 * tan-crimson-mite-blindfitter switched at 2020-09-06 00:00:00+00:00
 * intact-gray-marlin-trademarkagent switched at 2020-10-06 00:00:00+00:00
 * visual-copper-antlion-busconductor switched at 2020-02-18 00:00:00+00:00
 * japanese-yellow-pike-thermalengineer switched at 2020-02-17 00:00:00+00:00
 * basic-pink-fox-sharedealer switched at 2020-08-31 00:00:00+00:00
 * obedient-maroon-buzzard-caremanager switched at 2021-05-28 00:00:00+00:00
 * basic-blue-gull-physiotherapist switched at 2018-07-06 00:00:00+00:00
 * repulsive-violet-canidae-warehousewoman switched at 2018-07-06 00:00:00+00:00
 * uncertain-lavender-barracuda-ornithologist switched at 2018-11-22 00:00:00+00:00
 * green-green-piranha-nurserynurse switched at 2019-04-04 00:00:00+00:00
 * steady-jade-leech-payrollsupervisor switched at 2018-05-02 00:00:00+00:00
 * isolated-pink-dingo-nightwatchman switched at 2018-04-26 00:00:00+00:00
 * giant-azure-coyote-aeronauticalengineer switched at 2020-02-03 00:00:00+00